In [131]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
from datetime import date

In [132]:
data = pd.read_csv("data/df_new.csv")

In [133]:
data.Gender = pd.get_dummies(data.Gender, columns = ['Gender'], drop_first = True)
data = pd.merge(data, pd.get_dummies(data.Neighbourhood, columns = ['Neighbourhood'], drop_first = True), left_index=True, right_index=True)
#data = data.drop(columns=['Neighbourhood'])

### 1. Split data for optim part

In [134]:
df_opti = data[data.ScheduledTime >= "2016-05-30"]
df_opti = df_opti[df_opti.ScheduledTime <= "2016-06-05"]

df_opti.to_csv("data/data_opti.csv", index = False)

In [135]:
df_pred = data[data.ScheduledTime < "2016-05-30"]

In [136]:
df_train = df_pred[df_pred.AppointmentTime < "2016-05-30"]

df_train.to_csv("data/data_train.csv", index= False)

df_pred.to_csv("data/data_pred.csv", index = False)

### 2. Select one clinic and nb doctors

Select clinic with enough appointments and high no show rate : 5800 appointments and 21% no show for Maria Ortiz

In [137]:
neigh = "MARIA ORTIZ"

n_doc = np.floor(len(df_opti[df_opti.Neighbourhood == neigh])/(5*10))

In [9]:
n_doc = 22

In [10]:
df_pred_c = df_pred[df_pred[neigh] == 1]

In [11]:
df_opti_c = df_opti[df_opti[neigh] == 1]

### 3. Get prebooked slots

In [12]:
df_pred_c = df_pred_c[df_pred_c.AppointmentTime >= "2016-05-30"]
df_pred_c = df_pred_c[df_pred_c.AppointmentTime <= "2016-06-05"]
df_pred_c['AppointmentTime']= pd.to_datetime(df_pred_c['AppointmentTime'])
df_pred_c.AppointmentTime = df_pred_c.AppointmentTime.dt.tz_localize(None)
df_pred_c["slot"] = df_pred_c.apply(
    lambda row: (row.AppointmentTime - pd.to_datetime("2016-05-30")).days*10 + random.randint(0,9), axis=1)
df_pred_c.to_csv("data/pred_Maria.csv")

In [13]:
df_opti_c = df_opti_c[df_opti_c.AppointmentTime >= "2016-05-30"]
df_opti_c = df_opti_c[df_opti_c.AppointmentTime <= "2016-06-05"]
df_opti_c.to_csv("data/opti_Maria.csv")

In [14]:
df_opti_c

,PatientId,AppointmentID,Gender,ScheduledTime,AppointmentTime,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,SANTOS REIS,SEGURANÇA DO LAR,SOLON BORGES,SÃO BENEDITO,SÃO CRISTÓVÃO,SÃO JOSÉ,SÃO PEDRO,TABUAZEIRO,UNIVERSITÁRIO,VILA RUBIM
3250,23306,95781,0,2016-05-31 11:47:10+00:00,2016-05-31 23:59:59+00:00,49,MARIA ORTIZ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3266,38700,93903,0,2016-05-30 19:25:44+00:00,2016-05-31 23:59:59+00:00,24,MARIA ORTIZ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3294,41570,94241,0,2016-05-31 07:18:33+00:00,2016-05-31 23:59:59+00:00,42,MARIA ORTIZ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3296,9842,91782,0,2016-05-30 09:23:34+00:00,2016-05-30 23:59:59+00:00,52,MARIA ORTIZ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3297,40527,91640,0,2016-05-30 09:01:04+00:00,2016-05-30 23:59:59+00:00,2,MARIA ORTIZ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110499,59334,98085,0,2016-06-01 09:42:56+00:00,2016-06-01 23:59:59+00:00,66,MARIA ORTIZ,0,1,1,...,0,0,0,0,0,0,0,0,0,0
110501,5697,98260,1,2016-06-01 10:19:12+00:00,2016-06-01 23:59:59+00:00,44,MARIA ORTIZ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110505,37540,98410,0,2016-06-01 10:45:50+00:00,2016-06-01 23:59:59+00:00,55,MARIA ORTIZ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110506,9273,98531,1,2016-06-01 11:09:20+00:00,2016-06-01 23:59:59+00:00,5,MARIA ORTIZ,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 4. Generate random availability patient

In [99]:
for i in range(10):
    df_opti_c.loc[:,"Availability_"+str(i)] = [random.randint(0,49) for n in range(len(df_opti_c))]

df_opti_c.to_csv("data/opti_Maria.csv")

### 5. Generate day availability to compare models

In [100]:
df_opti_c['AppointmentTime']= pd.to_datetime(df_opti_c['AppointmentTime'])
df_opti_c.AppointmentTime = df_opti_c.AppointmentTime.dt.tz_localize(None)
for i in range(5):
    df_opti_c.loc[:,"Availability_day_"+str(i)] = df_opti_c.apply(
    lambda row: (row.AppointmentTime - pd.to_datetime("2016-05-30")).days*10 + random.randint(0,9), axis=1)

df_opti_c.to_csv("data/opti_Maria.csv")

### 6. Generate data for predictions (week)

In [101]:
df_opti_c.AppointmentYear.unique()

array([2016], dtype=int64)

In [102]:
map_matrix = np.zeros((50,3)) #day, month, weekday

In [103]:
map_matrix[:,0] = [30]*10 + [31]*10 + [1]*10 + [2]*10 + [3]*10
map_matrix[:,1] = [5]*20 + [6]*30
map_matrix[:,2] = [0]*10 + [1]*10 + [2]*10 + [3]*10 + [4]*10

In [104]:
newdf = pd.DataFrame(np.repeat(df_opti_c.values[:,:119], 10, axis=0))
newdf.columns = df_opti_c.columns[:119]
newdf["Hour_available"] = df_opti_c.iloc[:,109:119].to_numpy().flatten()

In [128]:
newdf["AppointmentDay"] = newdf["Hour_available"].map(lambda x: int(map_matrix[x,0]))
newdf["AppointmentMonth"] = newdf["Hour_available"].map(lambda x: int(map_matrix[x,1]))
newdf["AppointmentWeekDay"] = newdf["Hour_available"].map(lambda x: int(map_matrix[x,2]))
newdf["AwaitingTime"] = (newdf.AppointmentDay - newdf.ScheduledDay)%31

In [130]:
newdf.to_csv("data/df_new_week.csv", index = False)